# A linear algebra perspective on linear regression.

In this notebook, we focus on solving systems of equations by using linear algebra. In particular, we represent these systems of equations as matrices, and analyze properties of these matrices in order to understand when they have solutions. In the last part of this notebook, we connect these ideas to linear regression.

In [1]:
import numpy as np
np.set_printoptions(precision=2)

# Solving a system

Let's start with this simple system of equations and try to solve it by hand.

$$
\begin{align}
x_1 + x_2 + x_3 &= 3 \\
x_1 - x_2 + 2 x_3 &= 2 \\
x_2 + x_3 &= 2 \\
\end{align}
$$

*Exercise*: Try solving this system of equations. The answer is below.

1. We can solve for $x_1$ by subtracting the line 3 from line 1, so $x_1=1$.
2. We can plug that into line 2, so our system of equations is now

$$
\begin{align}
x_1 &= 1 \\
- x_2 + 2 x_3 &= 1 \\
x_2 + x_3 &= 2 \\
\end{align}
$$

3. We add line 3 to line 2. That leaves us with $3 x_3 = 3$, so $x_3 = 1$.
4. That means $x_2=1$! So our solution is $\mathbf{x} = \begin{bmatrix} 1 & 1 & 1 \end{bmatrix}$.

We just followed an ad-hoc algorithm to solve this matrix, but there are more systematic ways to solve it by 1) systematically repeating simplification steps (called Gaussian eliminiation) until 2) each row reaches a _canonical_ form (reduced row-echelon form). We won't cover this in any detail, but just point out that there are systematic algorithms that can do it, with preferable properties (efficiency, numerical stability).

*Fun fact:*
> *The method of Gaussian elimination appears – albeit without proof – in the Chinese mathematical text Chapter Eight: Rectangular Arrays of The Nine Chapters on the Mathematical Art. Its use is illustrated in eighteen problems, with two to five equations. The first reference to the book by this title is dated to 179 AD, but parts of it were written as early as approximately 150 BC.It was commented on by Liu Hui in the 3rd century.* [Wikipedia](https://en.wikipedia.org/wiki/Gaussian_elimination#History)

What do these systems of equations have in common with linear regression? They have some clear similarities (solving for linear weights). But also some clear differences (regression is noisy, we rarely have the same number of observations and features). The goal of these notes is to understand the linear algebra behind regression, in order to understand more about regression and when it does or doesn't have solutions.

# A more problematic system

Now, let's examine a different linear system to see some of the different kinds of solutions they can have.

$$
\begin{align}
x_1 + x_2 + x_3 &= 3 \\
x_1 - x_2 + 2 x_3 &= 2 \\
2 x_1 + 3 x_3 &= 1 \\
\end{align}
$$

If you add line 2 to line 1, you get $2 x_1 + 3 x_3 = 5$, which contradicts line 3! So, there's no way to solve this system of equations, making them inconsistent. But what happens if we change this to avoid that contradiction? If we only modify the last equation to avoid the contradiction, we get this system (change in bold)

$$
\begin{align}
x_1 + x_2 + x_3 &= 3 \\
x_1 - x_2 + 2 x_3 &= 2 \\
2 x_1 + 3 x_3 &= \textbf{5} \\
\end{align}
$$

From this, we can reduce to two equations, obtaining the first by subtracting line 2 from line 1

$$
\begin{align}
2 x_2 - x_3 &= 1 \\
2 x_1 + 3 x_3 &= 5 \\
\end{align}
$$

Any $\textbf{x}$ that satisfies these equations is a solution, like $\begin{bmatrix} 1 & 1 & 1 \end{bmatrix}$ or $\begin{bmatrix} -2 & 2 & 3 \end{bmatrix}$. In fact, there are infinitely many solutions!

In our example in the previous section, the system of equations seemed to have a single unique solution. In this example, depending on the $\textbf{b}$ on the right, its possible to have either zero or infinite solutions. In the coming sections, we'll try to analyze the differences of these matrices to explain these patterns of solutions.

## Linear systems as matrices

In this section, we write out some notation to relate the above systems of equations with the matrix equation $\textbf{A}\mathbf{x}=\mathbf{b}$.

First, some notation about matrices. Consider a matrix $\textbf{A}$ of size $n \times m$, which means it has $n$ rows and $m$ columns.
We can write it out in terms of the individual entries of the matrix. The entry at row $i$ and column $j$ is $a_{ij}$, so the matrix is

$$
\textbf{A} = \begin{bmatrix}
a_{11} & a_{12} & \ldots & a_{1m} \\
a_{21} & a_{22} & \ldots & a_{2m} \\
\ldots & \ldots & \ldots & \ldots \\
a_{n1} & a_{n2} & \ldots & a_{nm} \\
\end{bmatrix}
$$

Another way we can write it is in terms of the columns. The $i^{th}$ column is $\textbf{a}_i$, so the matrix is

$$
\textbf{A} = \begin{pmatrix} \textbf{a}_1 & \textbf{a}_2 & \ldots & \textbf{a}_m \end{pmatrix}
$$

So, if we go back to our above example equations, we can rewrite them using some of these notational ideas. Here's the system of equations from the first section:

$$
\begin{align}
x_1 + x_2 + x_3 &= 3 \\
x_1 - x_2 + 2 x_3 &= 2 \\
x_2 + x_3 &= 2 \\
\end{align}
$$

Each of these equations can correspond to a dot-product between $\textbf{x}$ and a vector with the factors (e.g., $\begin{bmatrix} 1 & 1 & 1 \end{bmatrix}$ for the first row). So, we can write this as a matrix equation

$$
\begin{bmatrix}
1 & 1 & 1 \\
1 & -1 & 2 \\
0 & 1 & 1 \\
\end{bmatrix} \mathbf{x} = \begin{bmatrix} 3 \\ 2 \\ 2 \end{bmatrix}
$$

Another different approach is to write it as a vector equation, using the columns of the above matrix.
$$
\begin{bmatrix} 1 \\ 1 \\ 0 \end{bmatrix} x_1 +
\begin{bmatrix} 1 \\ -1 \\ 1 \end{bmatrix} x_2 +
\begin{bmatrix} 1 \\ 2 \\ 1 \end{bmatrix} x_3
= \begin{bmatrix} 3 \\ 2 \\ 2 \end{bmatrix}
$$

All three of these representations are equivalent, and are all solved by $\textbf{x} = \begin{bmatrix} 1 & 1 & 1 \end{bmatrix}$. In future sections, we'll freely switch between these representations as necessary.

## Explaining the contradiction

Now, we're going to further analyze the above case where we reached a contradiction.
When we were solving system of equations above, we were trying to find a solution $\textbf{x}$ that satisfies the
the equation $\textbf{A}\textbf{x}=\textbf{b}$, given a specific matrix $\textbf{A}$ and vector $\textbf{b}$. From our above examples, a natural followup is to generalize to arbitrary $\textbf{b}$ by asking: Given a matrix $\textbf{A}$, can we say something general about the $\textbf{b}$ that will have a solution?

Let's look at our problematic example above, now in matrix form:
$$
\textbf{A} = \begin{bmatrix}
1 & 1 & 1 \\
1 & -1 & 2 \\
2 & 0 & 3 \\
\end{bmatrix}
$$

We can rewrite the equation $\textbf{A}\textbf{x}=\textbf{b}$ in a vectorized form as follows

$$
\textbf{a}_1 x_1 + \textbf{a}_2 x_2 + \ldots + \textbf{a}_m x_m = \textbf{b}
$$

Or, more specifically for our current example
$$
\begin{bmatrix} 1 \\ 1 \\ 2 \end{bmatrix} x_1 +
\begin{bmatrix} 1 \\ -1 \\ 0 \end{bmatrix} x_2 +
\begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix} x_3
= \textbf{b}
$$

This makes it clear that $\textbf{b}$ is a **linear combination** of the columns of $\textbf{A}$, weighted by $\textbf{x}$. A linear combination is a weighted sum of vectors. In this case, the weights are the elements of $\textbf{x}$, and the vectors that we combine are the columns of $\textbf{A}$. A related concept is the **span** of a generating set of vectors, which is the set of all possible linear combinations of the vectors. One final concept: The **column space** of a matrix, $col(\textbf{A})$ is the span of its columns. So, we can rephrase the above as: vectors $\textbf{b}$ with a solution must be in $col(\textbf{A})$, the column space of $\textbf{A}$.

So that explains part of what we saw above: we reached a contradiction with $\textbf{b} = \begin{bmatrix} 3 & 2 & 1 \end{bmatrix}$ because it cannot be expressed as a linear combination of the columns of $\textbf{A}$.

## Infinite solutions?

We have another property of our problematic matrix left to explain: the infinite number of solutions for certain $\textbf{x}$.
To explain this, we introduce the **null space**, or $null(\textbf{A})$, which are the solutions $\textbf{x'}$ so that $\textbf{A}\textbf{x'} = \textbf{0}$. Every matrix has the trivial solution $\textbf{x'}=\textbf{0}$, but only some matrices have other non-trivial solutions. Given a known solution $\textbf{x}$ and non-trivial element of the null space $\textbf{x'}$, you can construct another solution to by adding these equations together, since

$$
\begin{align}
\textbf{A}\textbf{x} + \textbf{A}\textbf{x'} &= \textbf{b} + \textbf{0} \\
\textbf{A}(\textbf{x} + \textbf{x'}) &= \textbf{b} \\
\end{align}
$$

So that means $\textbf{x}+\textbf{x'}$ is another solution. Since knowing the null space is a way to generate more solutions, let's see if we can solve $\textbf{A}\textbf{x} =\textbf{0}$ for our above matrices. We'll work with our problematic example in equation form.

*Exercise*: Solve $\textbf{A}\textbf{x} =\textbf{0}$.

$$
\begin{align}
x_1 + x_2 + x_3 &= 0 \\
x_1 - x_2 + 2 x_3 &= 0 \\
2 x_1 + 3 x_3 &= 0 \\
\end{align}
$$

If we subtract line 2 from line 1 like we did above, we get the following constraints

$$
\begin{align}
2 x_2 - x_3 &= 0 \\
2 x_1 + 3 x_3 &= 0 \\
\end{align}
$$

These constraints are satisfied by solutions like $\begin{bmatrix} -3 & 1 & 2 \end{bmatrix}$ or $\begin{bmatrix} -6 & 2 & 4 \end{bmatrix}$. So, it looks like the null space is the span of a set containing the vector $\begin{bmatrix} -3 & 1 & 2 \end{bmatrix}$.

And, if we look at the above case of infinite solutions ($\begin{bmatrix} 1 & 1 & 1 \end{bmatrix}$, $\begin{bmatrix} -2 & 2 & 3 \end{bmatrix}$), you'll notice that their difference is actually $\begin{bmatrix} -3 & 1 & 2 \end{bmatrix}$. This example shows that, given a solution $\textbf{x}$ to $\textbf{A}\textbf{x}=\textbf{b}$ and a non-trivial null space, you can produce an infinite number of solutions $\textbf{x} + \textbf{x}'$ for all $x' \in null(\textbf{A})$.

# A fundamental theorem

For our problematic matrix, we now have an explanation for cases where we reach the contradiction (the element is not in the span column space) and an explanation for cases where their infinite solutions (the null space is non-trivial, and can generate infinite solutions). These two properties of our problematic metrics are fundamentally related, which we'll explain now. But first, we'll introduce some helpful concepts.

A set of vectors is **linearly dependent** if one vector can be written as a linear combination of the others.
For example, let's look again at our problematic matrix:
$$
\textbf{A} = \begin{bmatrix}
1 & 1 & 1 \\
1 & -1 & 2 \\
2 & 0 & 3 \\
\end{bmatrix}
$$

We can write one column as a linear combination of the other two:
$$
\begin{bmatrix} 1 \\ -1 \\ 0 \end{bmatrix} =
3 \begin{bmatrix} 1 \\ 1 \\ 2 \end{bmatrix} -
2 \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix}
$$

*Exercise: How are the coefficients of this linear combination related to elements of the null space above? Why?*

So, our three columns are redundant! By removing the vector $\begin{bmatrix} 1 & -1 & 0 \end{bmatrix}$, we can remove this redundancy. After removing vectors that can be written as a linear combination of others, the remaining ones are **linearly independent**.

The redundancy in columns means that, even though we are working with three-dimensional vectors, our vectors are in a two-dimensional space defined by the linearly independent vectors. To keep these concepts separate, we introduce a new concept: the **dimensionality** of a space, $dim(\cdot)$. So our problemetic matrix $\textbf{A}$, which has a linearly independent set of two column vectors, has a two-dimensional column space, which we can write out as $dim(col(\textbf{A}))=2$.

We can now write the fundamental theorem of linear algebra

$$
size(\textbf{A}) = dim(col(\textbf{A})) + dim(null(\textbf{A}))
$$

where we have assumed a square matrix $\textbf{A}$ and the size of the matrix is $size(\cdot)$. The idea here is that there are only two possible cases for the result of $\textbf{A}\textbf{x}$, when $\textbf{x} \ne \textbf{0}$. It either results in $\textbf{0}$, so $\textbf{x} \in null(\textbf{A})$. Or, it results in a non-trivial $\textbf{b} \in col(\textbf{A})$.

One final definition: We give a special name to the dimensionality of the column space of a matrix $\textbf{A}$: the **rank** of the matrix. So, $rank(\textbf{A}) = dim(col(\textbf{A}))$. It's an important concept because it tells us something intrinsic about the matrix: how much information about $\textbf{x}$ is retained (i.e. mapped to the column space) by the result of $\textbf{A}\textbf{x}$. When a matrix has a rank that matches the dimensionality, so that $rank(\textbf{A})=size(\textbf{A})$, we call it **full-rank**.

## The unproblematic matrix

So, going back to our very first example, what was special about that system of equations? Let's try to compute its null space.

$$
\begin{align}
x_1 + x_2 + x_3 &= 0 \\
x_1 - x_2 + 2 x_3 &= 0 \\
x_2 + x_3 &= 0 \\
\end{align}
$$

We follow the same sequence of simplification steps as above, so

1. Subtract line 3 from line 1, so $x_1=0$.
2. Plug $x_1=0$ into line 2 and add line 3, so $3 x_3 = 0$.
3. Plug $x_3=0$ into line 3, so $x_2=0$.

So, the only solution to $\textbf{A}{x'}=\textbf{0}$ for this matrix is $\textbf{0}$. By the above fundamental theorem, that means $size(\textbf{A}) = dim(col(\textbf{A}))$, so our matrix is full-rank!


## Solving through inversion

Another approach we could take to solve $\textbf{A}\textbf{x}=\textbf{b}$ is to make use of the matrix inverse $\textbf{A}^{-1}$. The inverse is defined by the following relationship

$$
\textbf{A}^{-1} \textbf{A} = \textbf{I}
$$

With an inverse in hand, we can solve the above linear systems in a simple way

$$
\begin{align}
\textbf{A} \textbf{x} &= \textbf{b} \\
\textbf{A}^{-1} \textbf{A} \textbf{x} &= \textbf{A}^{-1} \textbf{b} \\
\textbf{I} \textbf{x} &= \textbf{A}^{-1} \textbf{b} \\
\textbf{x} &= \textbf{A}^{-1} \textbf{b} \\
\end{align}
$$

But which matrices have an inverse? Crucially, they require a trivial null space, where $\textbf{x}'=\textbf{0}$ is the only solution to $\textbf{A}\textbf{x'}=\textbf{0}$. Why? A non-trivial null space means there's no inverse that satisfies $\textbf{x'}=\textbf{A}^{-1} \textbf{0}$, since all the elements of $null(\textbf{A})$ are potential values for $\textbf{x'}$.

*In practice: Computing an inverse to solve a system of equations is usually computationally more expensive and can result in numerical issues, so it can be preferable to work directly with a linear system solver.*

*Fun Fact: We were able to use the same sequence of simplification steps to solve our equations as we changed the values of $\textbf{b}$. It turns out that simplifying a matrix in this way can also be adapted to invert matrices, and is related to certain matrix decompositions. Read more on [Wikipedia](https://en.wikipedia.org/wiki/Gaussian_elimination#Definitions_and_example_of_algorithm).*

## In practice: Solving the full-rank matrix

Here, we'll run code to solve our nicely behaved, full-rank matrix, showing our solutions match the above.

In [2]:
A = np.array([
    [1, 1, 1],
    [1, -1, 2],
    [0, 1, 1],
])
b = np.array([3, 2, 2])

Ainv = np.linalg.inv(A)
print('Checking our inverse works well. Should close to the identity:')
print(Ainv @ A)
print('Solution by inverse:', Ainv @ b)
print()

print('Solution by using a solver:', np.linalg.solve(A, b))

Checking our inverse works well. Should close to the identity:
[[ 1.00e+00 -2.22e-16  0.00e+00]
 [ 0.00e+00  1.00e+00 -5.55e-17]
 [ 0.00e+00  0.00e+00  1.00e+00]]
Solution by inverse: [1. 1. 1.]

Solution by using a solver: [1. 1. 1.]


# Regression

Keeping our symbols as above, linear regression is formulated as finding a minimum-error weight vector $\textbf{x}^*$ of size $m$ that can weight the columns of a feature matrix $\textbf{A}$ of size $n \times m$ to reproduce an expected output $\textbf{b}$ of size $n$. This matrix $\textbf{A}$ is a design matrix, with $n$ observations as rows and $m$ features as columns. Formally,

$$
\textbf{x}^* = \arg\min_x \|\textbf{A}\textbf{x} - \textbf{b}\|_2
$$

By taking a derivative to solve for the minimum of this expression (derivation below), we get the following equation, which resembles the simple approach to solving a linear system of equations we used above.

$$
\textbf{x}^* = (\textbf{A}^T\textbf{A})^{-1}\textbf{A}^T\textbf{b}
$$

This crucially depends on being able to take the inverse of $\textbf{A}^T\textbf{A}$, which is of size $m \times m$. For rectangular matrices, the maximum possible rank is $\min(m, n)$, so that gives us a few distinct cases to consider.

## When $n \ge m$, or more observations than features

Having more observations than features is the typical setting for linear regression.
When $n \ge m$, the matrix rank can be at most $m$, so $rank(\textbf{A}) \le m$.
If the features are linearly independent, then $rank(\textbf{A}) = m$, so $rank(\textbf{A}^T\textbf{A}) = m$, which makes it invertible, so we can compute the solution $\textbf{x}^*$ above. When $n > m$, we won't generally be able to find a solution so that $\textbf{A}\textbf{x}=\textbf{b}$, but our solution will have minimum error.

Let's think more about the geometry of these solutions. Here's an example matrix

$$
\textbf{A} = \begin{bmatrix}
1 & 0 \\
-1 & 0 \\
0 & 1 \\
0 & -1 \\
\end{bmatrix}
$$

We can find a perfect solution for some $\textbf{b}$, like $\begin{bmatrix} 2 & -2 & 3 & -3 \end{bmatrix}$ when $\textbf{x}=\begin{bmatrix} 2 & 3 \end{bmatrix}$. These $\textbf{b}$ with a perfect solution must be in $col(\textbf{A})$, and are unique since our columns are linearly independent. However, notice that our $\textbf{x}$ can have at most two elements -- this means we can only generate a two-dimensional output space, even though the size of $\textbf{b}$ is four! So, there are many $\textbf{b}$ that we _can't_ solve exactly, like $\begin{bmatrix} 7 & -6 & 3 & -4 \end{bmatrix}$. But, you can get a close solution $\textbf{x}=\begin{bmatrix} 6.5 & 3.5 \end{bmatrix}$. So, regression looks for an approximation of $\textbf{b}$ that's actually in $col(\textbf{A})$.

In [3]:
def solve(A, b):
    return np.linalg.inv(A.T @ A) @ A.T @ b

A = np.array([
    [1, 0],
    [-1, 0],
    [0, 1],
    [0, -1],
])

b = np.array([2, -2, 3, -3])
x = solve(A, b)
print(f'exact solution to {b=}: {x=} residuals={A@x-b}')

b = np.array([7, -6, 3, -4])
x = solve(A, b)
print(f'approximate solution to {b=}: {x=} residuals={A@x-b}')

exact solution to b=array([ 2, -2,  3, -3]): x=array([2., 3.]) residuals=[0. 0. 0. 0.]
approximate solution to b=array([ 7, -6,  3, -4]): x=array([6.5, 3.5]) residuals=[-0.5 -0.5  0.5  0.5]


However, things are different if our features are linearly dependent. In that case, $rank(\textbf{A}^T\textbf{A})<m$, so we can't invert the matrix $\textbf{A}^T\textbf{A}$ to compute a solution. Intuitively, it's no longer possible to find a unique solution, since some columns can be rewritten in terms of others. So even though we can solve some problems and approximate the rest, having linearly dependent columns means there are always infinite solutions, whether they are exact or approximate solutions.

A related issue can also occur with columns that are collinear, but not entirely linearly dependent. In particular, even when $\textbf{A}^T\textbf{A}$ can be inverted, correlated columns can make it difficult to interpret the weights $\textbf{x}^*$. For example, in a simple case with two nearly-identical columns, any combination of those two columns will result in similar error, which will cause estimates of their standard deviations to be large. While the model on the whole can still achieve low error, it might be difficult to understand which of the two columns is contributing most because of their relationship. Highly collinear columns can also cause numerical issues with inverting $\textbf{A}^T\textbf{A}$. More details are on [Wikipedia](https://en.wikipedia.org/wiki/Multicollinearity#Consequences), or in Cosma Shalizi's [course notes (in section 5)](https://www.stat.cmu.edu/~cshalizi/mreg/15/lectures/14/lecture-14.pdf).

*Exercise: We assumed $rank(\textbf{A}) = rank(\textbf{A}^T\textbf{A})$ above. Can you prove this? One approach is to show any element of $null(\textbf{A})$ is in $null(\textbf{A}^T \textbf{A})$, as well as the opposite. [Here's a proof](https://math.stackexchange.com/a/349966) that takes this approach.*

## When $n < m$, or more features than observations

This corresponds to cases where, intuitively, you would be overfitting data when fititng a regression model. We examine these issues by using linear algebra.

When $n < m$, the matrix rank can be at most $n$, so $rank(\textbf{A}) \le n$. But $size(\textbf{A}^T\textbf{A})$ is $m \times m$. That means that $rank(\textbf{A}^T\textbf{A}) \le n < m$, so $\textbf{A}^T\textbf{A}$ is not full rank, so we can't take an inverse. This also makes sense intuitively; having more features than observations means that some features will be linearly dependent, so there are infinite solutions. When $rank(\textbf{A}) < n$, it is also possible to have no solutions, since there are fewer linearly independent vectors than there are observations. Intuitively, we can think of these cases as analogous to our problematic matrix above.

For our example, let's consider a matrix really similar to the one above

$$
\textbf{A} = \begin{bmatrix}
1 & -1 & 0 & 0 \\
0 & 0 & 1 & -1 \\
\end{bmatrix}
$$

This matrix has linearly dependent columns (Can you give an example?), but does have a rank of $n=2$, since we have two linearly independent columns. So it has infinite solutions for any $\textbf{b}$, like $\begin{bmatrix} 3 & 4 \end{bmatrix}$, which is solved by $\textbf{x}=\begin{bmatrix} 3 & 0 & 4 & 0 \end{bmatrix} + c \begin{bmatrix} 1 & 1 & 1 & 1 \end{bmatrix}$ for any weight $c$.


## In practice: Solving the above examples

Here, we solve the various examples in this notebook. We sometimes have different results between our simple least squares solution, and `np.linalg.lstsq`. This happens because `np.linalg.lstsq` uses a more robust inverse than our simple approach.

In [4]:
examples = [
    dict(
        name='good',
        A=np.array([
            [1, 1, 1],
            [1, -1, 2],
            [0, 1, 1],
        ]),
        b=np.array([3, 2, 2]),
    ),
    dict(
        name='bad-infinite',
        A=np.array([
            [1, 1, 1],
            [1, -1, 2],
            [2, 0, 3],
        ]),
        b=np.array([3, 2, 5]),
    ),
    dict(
        name='bad-contradiction',
        A=np.array([
            [1, 1, 1],
            [1, -1, 2],
            [2, 0, 3],
        ]),
        b=np.array([3, 2, 1]),
    ),
    dict(
        name='linreg-exact',
        A=np.array([
            [1, 0],
            [-1, 0],
            [0, 1],
            [0, -1],
        ]),
        b=np.array([2, -2, 3, -3]),
    ),
    dict(
        name='linreg-approx',
        A=np.array([
            [1, 0],
            [-1, 0],
            [0, 1],
            [0, -1],
        ]),
        b=np.array([7, -6, 3, -4]),
    ),
    dict(
        name='linreg-linear-dependence',
        A=np.array([
            [1, 0, 1],
            [-1, 0, -1],
            [0, 1, 1],
            [0, -1, -1],
        ]),
        b=np.array([7, -6, 3, -4]),
    ),
    dict(
        name='linreg-overfitting',
        A=np.array([
            [1, -1, 0, 0],
            [0, 0, 1, -1],
        ]),
        b=np.array([3, 4]),
    ),
]

def solver_inv(A, b):
    Ainv = np.linalg.inv(A)
    assert np.allclose(np.eye(A.shape[0]), Ainv @ A), f'Invalid inverse, should be identity: {Ainv@A=}'
    return Ainv @ b

def solver_solve(A, b):
    return np.linalg.solve(A, b)

def solver_lstsq_simple(A, b):
    ATA = A.T @ A
    ATAinv = np.linalg.inv(ATA)
    assert np.allclose(np.eye(ATA.shape[0]), ATAinv @ ATA), f'Invalid inverse, should be identity: {ATAinv @ ATA=}'
    return ATAinv @ A.T @ b

def solver_lstsq(A, b):
    x, resid, rank, singular = np.linalg.lstsq(A, b, rcond=1e-5)
    print(f'\t{rank=}')
    return x

for example in examples:
    A = example['A']
    b = example['b']

    print()
    print('-' * 10)
    print()
    print('example =', example['name'])
    print(f'\t{A=}')
    print(f'\t{b=}')

    for solver in [solver_inv, solver_solve, solver_lstsq_simple, solver_lstsq]:
        try:
            print(f'| Solving with {solver.__name__}...')
            x = solver(A, b)
            resid = A@x-b
            print(f'\t{x=}  |  approx b={A@x=}  |  residuals={resid}  |  sum squared error={resid@resid:.2f}')
        except Exception as err:
            print(f'\tCould not solve with {solver.__name__}. Error:', err)


----------

example = good
	A=array([[ 1,  1,  1],
       [ 1, -1,  2],
       [ 0,  1,  1]])
	b=array([3, 2, 2])
| Solving with solver_inv...
	x=array([1., 1., 1.])  |  approx b=A@x=array([3., 2., 2.])  |  residuals=[-4.44e-16 -4.44e-16 -2.22e-16]  |  sum squared error=0.00
| Solving with solver_solve...
	x=array([1., 1., 1.])  |  approx b=A@x=array([3., 2., 2.])  |  residuals=[0. 0. 0.]  |  sum squared error=0.00
| Solving with solver_lstsq_simple...
	x=array([1., 1., 1.])  |  approx b=A@x=array([3., 2., 2.])  |  residuals=[ 1.78e-15  1.33e-15 -4.44e-16]  |  sum squared error=0.00
| Solving with solver_lstsq...
	rank=3
	x=array([1., 1., 1.])  |  approx b=A@x=array([3., 2., 2.])  |  residuals=[ 4.44e-16 -4.44e-16 -4.44e-16]  |  sum squared error=0.00

----------

example = bad-infinite
	A=array([[ 1,  1,  1],
       [ 1, -1,  2],
       [ 2,  0,  3]])
	b=array([3, 2, 5])
| Solving with solver_inv...
	Could not solve with solver_inv. Error: Singular matrix
| Solving with solver_solve.

## Appendix: Regression derivation

We briefly derive the above expression for regression, using somewhat informal notation for our derivatives. We want to minimize the squared error of the model's predictions:

$$
\begin{align}
\mathcal{L}(\textbf{x}) &= \|\textbf{A}\textbf{x} - \textbf{b}\|^2_2 \\
&= (\textbf{A}\textbf{x} - \textbf{b})^T(\textbf{A}\textbf{x} - \textbf{b}) \\
&= \textbf{x}^T \textbf{A}^T \textbf{A}\textbf{x} - 2 \textbf{b}^T\textbf{A}\textbf{x} + \textbf{b}^T\textbf{b}
\end{align}
$$

Now, we take the derivative of this loss, with respect to our solution vector $\textbf{x}$. Since we're taking a derivative with respect to a vector, we'll need to use some multivariate, or matrix, calculus. We start with a few identities that we'll need below (pulled from this [cheat sheet](http://www.gatsby.ucl.ac.uk/teaching/courses/sntn/sntn-2017/resources/Matrix_derivatives_cribsheet.pdf)), which resemble the derivatives you'd take in univariate calculus. The vector $\textbf{c}$ and matrix $\textbf{C}$ are constants.

$$
\begin{align}
\frac{d}{d \textbf{x}} \textbf{c} &= \textbf{0} \\
\frac{d}{d \textbf{x}} \textbf{x}^T \textbf{c} &= \textbf{c} \\
\frac{d}{d \textbf{x}} \textbf{x}^T \textbf{C} \textbf{x} &= 2 \textbf{C} \textbf{x} \\
\end{align}
$$

With those identities in place, we can take the derivative.

$$
\begin{align}
\frac{d}{d \textbf{x}} \mathcal{L}(x) 
&= \frac{d}{d \textbf{x}} \left[ \textbf{x}^T \textbf{A}^T \textbf{A}\textbf{x} - 2 \textbf{x}^T\textbf{A}^T\textbf{b} + \textbf{b}^T\textbf{b} \right] \\
&= 2 \textbf{A}^T \textbf{A} \textbf{x} - 2 \textbf{A}^T\textbf{b} \\
% &= 2 \textbf{x}^T \textbf{A}^T \textbf{A} - 2 \textbf{b}^T\textbf{A} \\
\end{align}
$$

Now, we use this to solve for a minimum error solution by setting the derivative of the loss to zero.
$$
\begin{align}
0 &= 2 \textbf{A}^T \textbf{A} \textbf{x}^* - 2 \textbf{A}^T\textbf{b} \\
\textbf{A}^T \textbf{A} \textbf{x}^* &= \textbf{A}^T\textbf{b} \\
\textbf{x}^* &= (\textbf{A}^T \textbf{A})^{-1} \textbf{A}^T\textbf{b} \\
\end{align}
$$

# References

- Inspiration and examples taken liberally from *Mathematics for Machine Learning* by Deisenroth, Faisal, and Ong.
- *Linear Algebra and its Applications* by Lay
- *Linear Algebra Done Right* by Axler
- *Introduction to Applied Linear Algebra* by Boyd and Vandenberghe